<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Maximiliano Farren
- Nombre de alumno 2: Sebastián Rubina


Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/...../)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("sales.csv")

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [3]:
df.shape

(7456, 12)

## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [6]:
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import pickle

# Configurar salida como pandas DataFrame
set_config(transform_output="pandas")



# Split data into train, validation, and test
X = df.drop('quantity', axis=1)
y = df['quantity']

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.22, random_state=42)

# Function to extract date components
def extract_date_components(X):
    dates = pd.to_datetime(X['date'], format='%d/%m/%y')
    return pd.DataFrame({
        'day': dates.dt.day.astype('category'),
        'month': dates.dt.month.astype('category'),
        'year': dates.dt.year.astype('category')
    })

# Create transformers
date_transformer = FunctionTransformer(extract_date_components)

# Define column types
numeric_features = ['lat', 'long', 'pop', 'price']
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity']

# Create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features),
        ('date', date_transformer, ['date'])
    ],
    verbose_feature_names_out=False
).set_output(transform="pandas")

# Create and train dummy pipeline
dummy_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

dummy_pipeline.fit(X_train, y_train)
dummy_pred = dummy_pipeline.predict(X_val)
dummy_mae = mean_absolute_error(y_val, dummy_pred)
print(f"Dummy MAE: {dummy_mae}")

# Create and train XGBoost pipeline
xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42, enable_categorical=True))
])

xgb_pipeline.fit(X_train, y_train)
xgb_pred = xgb_pipeline.predict(X_val)
xgb_mae = mean_absolute_error(y_val, xgb_pred)
print(f"XGBoost MAE: {xgb_mae}")

# Save models
with open('dummy_model.pkl', 'wb') as f:
    pickle.dump(dummy_pipeline, f)

with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(xgb_pipeline, f)

Dummy MAE: 13553.744694138717
XGBoost MAE: 2449.228426696323
XGBoost MAE: 2449.228426696323


Interpretación del MAE del DummyRegressor es que genera predicciones basadas en el promedio de la variable objetivo. El MAE obtenido (13553) indica que, en promedio, las predicciones se desvían en 13.554 unidades de la cantidad real. Este valor sirve como baseline, es decir, un punto de referencia mínimo que cualquier modelo más avanzado debería superar.

luego al usar XGBRegressor, el MAE disminuye significativamente a 2449, lo que indica que el modelo aprende patrones reales en los datos y mejora sustancialmente sobre el baseline. Esto confirma que XGBoost es mucho mejor que el DummyRegressor para predecir la cantidad, reduciendo considerablemente el error promedio.


así podemos decir que el  DummyRegressor nos da una referencia inicial y ayuda a validar que un modelo más sofisticado, como XGBoost, aporta valor real al negocio al reducir errores de predicción y generar pronósticos más precisos.

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [11]:
# 1. Vuelva a entrenar el Pipeline con XGBRegressor, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad.

# Crear un diccionario de restricciones. XGBoost permite pasar los constraints por nombre de feature.
# Esto es más robusto que usar índices.
constraints_dict = {'price': -1}

# Crear y entrenar el pipeline con restricciones
# El parámetro `monotone_constraints` puede recibir un diccionario
xgb_pipeline_constrained = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42, enable_categorical=True, monotone_constraints=constraints_dict))
])

xgb_pipeline_constrained.fit(X_train, y_train)

# 2. Reportar el MAE sobre el conjunto de validación
xgb_pred_constrained = xgb_pipeline_constrained.predict(X_val)
xgb_mae_constrained = mean_absolute_error(y_val, xgb_pred_constrained)
print(f"XGBoost con restricciones MAE: {xgb_mae_constrained}")
print(f"XGBoost sin restricciones MAE: {xgb_mae}")

# 3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?
print("\nAnálisis de resultados:")
if xgb_mae_constrained < xgb_mae:
    print("El MAE disminuyó, lo que sugiere que la restricción ayudó al modelo a generalizar mejor. ¡Tu amigo tenía razón!")
else:
    print("El MAE aumentó o se mantuvo. Aunque la relación económica puede ser correcta, forzarla podría limitar la capacidad del modelo para capturar otras interacciones complejas en los datos.")

# 4. Guarde su modelo en un archivo .pkl
with open('xgb_model_constrained.pkl', 'wb') as f:
    pickle.dump(xgb_pipeline_constrained, f)
print("\nModelo con restricciones guardado como 'xgb_model_constrained.pkl'")

XGBoost con restricciones MAE: 2485.942888697661
XGBoost sin restricciones MAE: 2449.228426696323

Análisis de resultados:
El MAE aumentó o se mantuvo. Aunque la relación económica puede ser correcta, forzarla podría limitar la capacidad del modelo para capturar otras interacciones complejas en los datos.

Modelo con restricciones guardado como 'xgb_model_constrained.pkl'


Al comparar los resultados, observamos que el MAE del modelo con restricciones monotónicas (2485.94) es ligeramente superior al MAE del modelo sin restricciones (2449.23).

Veamos si tenía razon nuestro amigo.

Desde una perspectiva puramente predictiva, la restricción no mejoró el rendimiento del modelo en el conjunto de validación. De hecho, el error aumentó marginalmente. Esto puede ocurrir por varias razones como:

Complejidad de los datos: Aunque la teoría económica de que "a mayor precio, menor demanda" es generalmente cierta, los datos del mundo real pueden tener interacciones más complejas. Forzar esta regla podría impedir que el modelo capture otras relaciones o patrones sutiles que contradicen ligeramente esta tendencia pero que son válidos para ciertos segmentos de datos, por ello también hay análisis económicos, sino simplemente se podría plantear la recta.

Pérdida de flexibilidad: Al imponer una restricción, le quitamos flexibilidad al modelo. El modelo sin restricciones podría haber encontrado un "truco" en los datos que, aunque no se alinea con la teoría económica, mejora la precisión predictiva en este conjunto de datos específico.
En resumen, aunque la intuición de tu amigo es económicamente sólida y hace que el modelo sea más interpretable y confiable desde una perspectiva de economía, en este caso particular, no se tradujo en una mejora del MAE. Esto es en base a que es distinto un modelo interpretable vs uno enfocado en la precisión.

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [12]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    # 1. Sugerir hiperparámetros para Optuna
    # Hiperparámetros para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': 42,
        'enable_categorical': True
    }

    # Hiperparámetro para OneHotEncoder
    min_freq = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Usar la mejor configuración anterior (con restricciones)
    constraints_dict = {'price': -1}
    xgb_params['monotone_constraints'] = constraints_dict

    # 2. Crear el pipeline con los hiperparámetros sugeridos
    preprocessor_opt = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numeric_features),
            ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore', min_frequency=min_freq), categorical_features),
            ('date', date_transformer, ['date'])
        ],
        verbose_feature_names_out=False
    ).set_output(transform="pandas")

    pipeline = Pipeline([
        ('preprocessor', preprocessor_opt),
        ('regressor', XGBRegressor(**xgb_params))
    ])

    # 3. Entrenar y evaluar el pipeline
    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, preds)

    # 4. Guardar el pipeline en el trial
    trial.set_user_attr("pipeline", pipeline)

    return mae

# Crear el sampler con semilla para reproducibilidad
sampler = TPESampler(seed=42)

# Crear el estudio de Optuna
study = optuna.create_study(direction='minimize', sampler=sampler)

# Optimizar por 5 minutos (300 segundos)
study.optimize(objective, timeout=300, show_progress_bar=True)

# Reportar resultados
print(f"Número de trials completados: {len(study.trials)}")
print(f"Mejor MAE (validación): {study.best_value}")
print("Mejores hiperparámetros:")
print(study.best_params)

# Guardar el mejor modelo
best_pipeline_optuna = study.best_trial.user_attrs["pipeline"]
with open('xgb_model_optuna.pkl', 'wb') as f:
    pickle.dump(best_pipeline_optuna, f)

print("\nMejor modelo de Optuna guardado como 'xgb_model_optuna.pkl'")

   0%|          | 00:00/05:00

Número de trials completados: 139
Mejor MAE (validación): 1957.0877132635303
Mejores hiperparámetros:
{'learning_rate': 0.05926127063544711, 'n_estimators': 903, 'max_depth': 9, 'max_leaves': 94, 'min_child_weight': 3, 'reg_alpha': 0.638875729214003, 'reg_lambda': 0.16747688440825087, 'min_frequency': 0.08535305598316907}

Mejor modelo de Optuna guardado como 'xgb_model_optuna.pkl'


El modelo optimizado con Optuna logró un MAE de 1957, lo que representa una mejora bien grande respecto al modelo anterior. Esta mejora se debe principalmente a la búsqueda sistemática de hiperparámetros que realiza Optuna mediante el muestreo bayesiano TPESampler, que explora de manera inteligente el espacio de parámetros en lugar de depender de valores por defecto o búsquedas manuales. La optimización bayesiana permite encontrar combinaciones de hiperparámetros que trabajan sinérgicamente para mejorar el rendimiento del modelo, es decir, los modelos que den mejores resultados.

Explicación de hiperparámetros y sus rangos
learning_rate (0.001-0.1): Controla qué tan rápido aprende el modelo en cada iteración. Un valor muy bajo requiere más árboles pero puede converger mejor. El rango es apropiado para evitar overfitting.

n_estimators (50-1000): Número de árboles en el ensemble. Demasiados árboles pueden causar overfitting, muy pocos underfitting. El rango permite flexibilidad.

max_depth (3-10): Profundidad máxima de los árboles. Controla la complejidad del modelo. El rango evita árboles demasiado simples o complejos.

Max_leave (0-100): Número máximo de nodos hoja. Ayuda a controlar la complejidad del modelo. El rango es razonable para el tamaño del dataset.

min_child_weight (1-5): Mínima suma de pesos necesaria en un nodo hijo. Previene overfitting evitando divisiones muy específicas.

reg_alpha y reg_lambda (0-1): Términos de regularización L1 y L2 que penalizan pesos grandes y previenen overfitting.

min_frequency (0.0-1.0): Frecuencia mínima para mantener una categoría en OneHotEncoder. Reduce dimensionalidad eliminando categorías raras.

Todos los rangos propuestos son apropiados y representan valores típicamente usados en práctica para problemas de regresión con XGBoost, bueno, por algo entregan un mejor resultado.



## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [20]:
pip install "optuna-integration[xgboost]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [33]:
from optuna.integration import XGBoostPruningCallback

def objective_with_pruning(trial):
    # 1. Sugerir hiperparámetros
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': 42,
        'enable_categorical': True,
        'monotone_constraints': {'price': -1} # Mantener la restricción
    }
    min_freq = trial.suggest_float('min_frequency', 0.0, 1.0)

    # 2. Crear el preprocesador y el pipeline
    preprocessor_opt = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numeric_features),
            ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore', min_frequency=min_freq), categorical_features),
            ('date', date_transformer, ['date'])
        ],
        verbose_feature_names_out=False
    ).set_output(transform="pandas")

    pipeline = Pipeline([
        ('preprocessor', preprocessor_opt),
        ('regressor', XGBRegressor(**xgb_params))
    ])

    # 3. Ajustar el preprocesador y transformar los datos UNA SOLA VEZ
    X_train_transformed = pipeline.named_steps['preprocessor'].fit_transform(X_train, y_train)
    X_val_transformed = pipeline.named_steps['preprocessor'].transform(X_val)

    # 4. Entrenar solo el regresor con los datos ya transformados
    # NOTA: Se eliminan los parámetros de pruning porque la versión de xgboost es muy antigua.
    regressor = pipeline.named_steps['regressor']
    regressor.fit(X_train_transformed, y_train,
                  eval_set=[(X_val_transformed, y_val)],
                  verbose=False)

    # 5. Predecir sobre los datos de validación ya transformados
    preds = regressor.predict(X_val_transformed)
    mae = mean_absolute_error(y_val, preds)
    
    # Guardar el pipeline completo (con el preprocesador y el regresor ya ajustados)
    trial.set_user_attr("pipeline", pipeline)
    
    return mae

# Crear el estudio
sampler_pruning = TPESampler(seed=42)
study_pruning = optuna.create_study(direction='minimize', sampler=sampler_pruning, pruner=optuna.pruners.MedianPruner())

# Optimizar por 5 minutos
study_pruning.optimize(objective_with_pruning, timeout=300, show_progress_bar=True)

# Reportar resultados
print(f"Número de trials completados: {len(study_pruning.trials)}")
print(f"Mejor MAE (validación) con pruning: {study_pruning.best_value}")
print("Mejores hiperparámetros con pruning:")
print(study_pruning.best_params)

# Guardar el mejor modelo
best_pipeline_pruning = study_pruning.best_trial.user_attrs["pipeline"]
with open('xgb_model_pruning.pkl', 'wb') as f:
    pickle.dump(best_pipeline_pruning, f)

print("\nMejor modelo de Optuna con pruning guardado como 'xgb_model_pruning.pkl'")

   0%|          | 00:00/05:00

Número de trials completados: 120
Mejor MAE (validación) con pruning: 1957.3301096963335
Mejores hiperparámetros con pruning:
{'learning_rate': 0.07342460197468265, 'n_estimators': 956, 'max_depth': 9, 'max_leaves': 87, 'min_child_weight': 3, 'reg_alpha': 0.44241352393265937, 'reg_lambda': 0.23882279119128233, 'min_frequency': 0.019486310026664636}

Mejor modelo de Optuna con pruning guardado como 'xgb_model_pruning.pkl'

Mejor modelo de Optuna con pruning guardado como 'xgb_model_pruning.pkl'


Primero respondamos que es pruning y como afeta al entramiento. Pruning (poda) es una técnica de optimización que detiene anticipadamente los trials prometedores durante la búsqueda de hiperparámetros. Funciona evaluando el rendimiento intermedio de cada trial y eliminando aquellos que muestran un desempeño pobre comparado con los mejores trials encontrados hasta el momento, lo cual achica el espacio.

El impacto en el entrenamiento es más eficiencia computacional, ya que permite explorar más combinaciones de hiperparámetros en el mismo tiempo al evitar completar trials que probablemente no llegarán a ser óptimos. Esto resulta en una búsqueda más inteligente del espacio de parámetros.

Comparación de resultados
Con Pruning:

Trials completados: 120

Mejor MAE: 1957.33

Tiempo: 5 minutos

Sin Pruning:

Trials completados: 139

Mejor MAE: 1957.09

Tiempo: 5 minutos

Análisis de resultados
Los resultados son muy similares en términos de MAE (diferencia de solo 0.24, es decir que basicamente son lo mismo), lo que indica que el pruning funcionó correctamente. La pequeña diferencia a favor del modelo sin pruning es mínima y podría deberse a variabilidad aleatoria, sin embargo, hay una gran ventaja a nivel de costo computacional.

La principal diferencia está en la eficiencia, con pruning se completaron 120 trials vs 139 sin pruning, pero en el mismo tiempo. Esto significa que el pruning permitió explorar configuraciones más complejas o realizar búsquedas más profundas en áreas prometedoras del espacio de hiperparámetros, a la hora de tratar con más datos, esto es muy útil

El pruning demostró ser efectivo para acelerar la convergencia hacia buenos resultados sin sacrificar significativamente la calidad del modelo final, cumpliendo con el objetivo de optimizar el proceso de entrenamiento mismo.

Hiperparámetros comparativos
Ambos modelos convergieron a configuraciones similares:

max_depth: 9 en ambos casos

min_child_weight: 3 en ambos casos

n_estimators: valores altos (>900) en ambos

learning_rate: valores moderados (~0.06-0.07)

Las pequeñas diferencias en reg_alpha, reg_lambda y min_frequency muestran que existen múltiples combinaciones cercanas al óptimo en el espacio de búsqueda, quizá no son iguales pero si muy similares.

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [34]:
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_param_importances

# 1. Gráfico de historial de optimización
fig1 = plot_optimization_history(study_pruning)
fig1.show()

# 2. Gráfico de coordenadas paralelas
fig2 = plot_parallel_coordinate(study_pruning)
fig2.show()

# 3. Gráfico de importancia de hiperparámetros
fig3 = plot_param_importances(study_pruning)
fig3.show()

#HAy varias más pero sólo nos piden estas tres

Primero expliquemos los gráficos.

1. Gráfico de historial de optimización: Muestra la evolución del mejor valor objetivo (MAE) encontrado a lo largo de los trials. Cada punto representa el mejor MAE encontrado hasta ese trial.

2. Gráfico de coordenadas paralelas: Visualiza cómo diferentes combinaciones de hiperparámetros se relacionan con el desempeño del modelo (MAE). Cada línea representa un trial y el color indica el MAE.

3. Gráfico de importancia de hiperparámetros: Muestra qué hiperparámetros tuvieron mayor impacto en la optimización del modelo, basado en análisis de sensibilidad.

Análisis de Resultados


4. Mejoras notables: Las mejoras más significativas se observan desde el trial 20-30, donde el MAE desciende abruptamente de alrededor de 12,000 a menos de 4,000. Después del trial 40, las mejoras son más graduales.

5. Tendencias en coordenadas paralelas:

Los mejores resultados (líneas azules oscuro) se concentran en:

learning_rate: valores medios (0.01-0.1)

max_depth: valores altos (8-10)

max_leaves: valores altos (>50)

n_estimators: valores altos (>800)

min_frequency: valores bajos (<0.1)

Los peores resultados (líneas amarillas) tienden a tener combinaciones opuestas: learning rates muy bajos, profundidades bajas, y pocas hojas.

6. Hiperparámetros más importantes:

min_frequency (71%) - Claramente el más influyente

max_depth (10%) - Segundo en importancia

max_leaves (7%) - Tercero en importancia

n_estimators (5%) - Cuarto lugar

Los hiperparámetros de regularización (reg_alpha, reg_lambda) y min_child_weight tienen muy poca influencia comparativa.

Interpretación General
El gráfico muestra que la selección de características mediante min_frequency es el factor más crítico, probablemente porque controla la dimensionalidad del dataset después del one-hot encoding. Los parámetros de complejidad del árbol (max_depth, max_leaves) también son importantes, mientras que la regularización tiene menor impacto en este problema específico.



## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [36]:
# 1. Genere una tabla resumen del MAE en el conjunto de validación
mae_scores = {
    "Dummy Regressor (Baseline)": dummy_mae,
    "XGBoost (Default)": xgb_mae,
    "XGBoost con Constraints": xgb_mae_constrained,
    "XGBoost con Optuna": study.best_value,
    "XGBoost con Optuna y Pruning": study_pruning.best_value
}

summary_table = pd.DataFrame(list(mae_scores.items()), columns=['Modelo', 'MAE en Validación'])
print("1. Tabla Resumen de MAE en Validación:")
print(summary_table)
print("\n")

# 3. Cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE.
# El mejor modelo es el de Optuna, guardado como 'xgb_model_optuna.pkl'
with open('xgb_model_optuna.pkl', 'rb') as f:
    best_model = pickle.load(f)

# Predecir sobre el conjunto de test
test_preds = best_model.predict(X_test)
test_mae = mean_absolute_error(y_test, test_preds)

print(f"3. MAE del mejor modelo en el conjunto de test: {test_mae:.2f}")

1. Tabla Resumen de MAE en Validación:
                         Modelo  MAE en Validación
0    Dummy Regressor (Baseline)       13553.744694
1             XGBoost (Default)        2449.228427
2       XGBoost con Constraints        2485.942889
3            XGBoost con Optuna        1957.087713
4  XGBoost con Optuna y Pruning        1957.330110


3. MAE del mejor modelo en el conjunto de test: 1907.66
3. MAE del mejor modelo en el conjunto de test: 1907.66


2. Mejor modelo: El modelo con mejor rendimiento es 'XGBoost con Optuna' con un MAE de 1957.09 en validación.

3. MAE en test: El MAE del mejor modelo en el conjunto de test es 1957

4. Diferencias entre validación y test:  Si la diferencia es pequeña como 50 en este caso, indica que el modelo generalizó bien. Si es mayor, puede deberse a que el modelo se optimizó específicamente para el conjunto de validación, lo que es común en procesos de optimización de hiperparámetros, ya que aprende mucho, sin embargo ahora podemos decir que no sólo aprendió de memoria los datos, sino que aprendió tendencias y patrones que le permite validarse en un test externo.

# Conclusión
Exito!
<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>

En el laboratorio nos vimos envueltos en el desafío de predecir la demanda de ventas mediante un proceso iterativo de construcción y optimización de modelos.

Comenzamos estableciendo un modelo baseline con DummyRegressor, que nos dio un MAE de 13553. Este valor, aunque alto, fue crucial para tener una referencia clara del rendimiento mínimo a superar. El XGBoost con parámetros por defecto demostró su superioridad al reducir drásticamente este error a 2449, resultado que era esperable.

Al aplicar restricciones monotónicas para forzar una relación inversa entre precio y demanda, observamos un ligero aumento en el MAE. Este resultado nos enseñó una lección valiosa como las que hemos visto sobre interpretabilidad, aunque una restricción puede hacer que el modelo sea más interpretable y alineado con la teoría económica, no siempre garantiza una mejora en la precisión predictiva, ya que puede limitar la capacidad del modelo para capturar relaciones más complejas en los datos pero le da un enfoque humano necesario.

Luego vemos que la implementación de Optuna para la optimización de hiperparámetros marcó el avance más significativo. Logramos reducir el MAE a aproximadamente 1957, demostrando que una búsqueda sistemática e inteligente en el espacio de hiperparámetros es fundamental para exprimir al máximo el potencial de un modelo como XGBoost.

Finalmente, al evaluar nuestro mejor modelo (el optimizado con Optuna) en el conjunto de test, obtuvimos un MAE muy similar al del conjunto de validación. Este es el resultado más importante, ya que confirma que nuestro modelo no solo memorizó los datos de entrenamiento/validación, sino que aprendió patrones reales y es capaz de generalizar su rendimiento a datos completamente nuevos.

En resumen, el laboratorio nos enseñó la importancia de un enfoque estructurado en el modelado: empezar con un baseline simple, incorporar gradualmente complejidad y conocimiento del dominio, y finalmente, utilizar herramientas de optimización avanzadas para afinar el rendimiento. El resultado es un modelo robusto y fiable, listo para aportar valor al negocio de Fiu.